In [1]:
structures = [:none, :hous, :scho, :shop, :offi, :manu]
cost = Dict(zip(structures, [0 2 4 3 9 7]))
eco_out = Dict(zip(structures, [0 0 -2 2 6 7]))
attract = Dict(zip(structures, [0 2 6 2 1 -1]))

M = 10 # Height of Array
N = 10 # Length of Array

# Starting map
noneStart = fill(1, M, N)
housStart = fill(0, M, N)
schoStart = fill(0, M, N)
shopStart = fill(0, M, N)
offiStart = fill(0, M, N)
manuStart = fill(0, M, N)

initial_map = Dict(zip(structures, [noneStart, housStart, schoStart, shopStart, offiStart, manuStart])) # fill(:none, N, M)

# Edge Values
# noneEV = Dict(zip(structures, [0 0 -10 -5  0  5]))
# housEV = Dict(zip(structures, [0 1   1  0 -1 -2]))


budget = 500
pop = 200

# println(initial_map)
;

In [3]:
function printMap(map) # List of 1's and zeros for each structure
    real_map = fill(:none, M, N)
    for s in structures
        for i = 1:M
            for j = 1:N
                if map[s, i, j] > 0
                    real_map[i, j] = s 
                end
            end
        end
    end

    for i = 1:M
        for j = 1:N
            print(real_map[i, j], "\t")
        end
        println()
    end
end
;

printMap (generic function with 1 method)

In [4]:
using JuMP, Cbc

m = Model(solver = CbcSolver())

# Variables
# Each structure keeps track of its own buildings and where they are. i = x coord while j = y coord
@variable(m, objMap[structures, 1:M, 1:N], Bin)
# Indicates change from prior map
@variable(m, change[structures, 1:M, 1:N], Bin) 
# Amount spent on stuff
@variable(m, spending[1:M, 1:N] >= 0) 
# Economic Output of area
@variable(m, ecoOut[1:M, 1:N]) 
# Attractiveness of the area
@variable(m, attrac[1:M, 1:N]) 

# Constraints

# Can't go overbudget
@constraint(m, sum(spending) <= budget)

for i = 1:M
    for j = 1:N
        
        # Buildings cannot share same location
        @constraint(m, sum(objMap[s, i, j] for s in structures) == 1)
#         end
        
        # Change happens if they are diferent. Constrained for absolute value
        for s in structures
                @constraint(m, objMap[s, i, j] - initial_map[s][i, j] <= change[s, i, j] )
                @constraint(m, objMap[s, i, j] - initial_map[s][i, j] >= -change[s, i, j] )
        end # Needed
        
        # Spending is based on new buildings
        @constraint(m, spending[i, j] == sum(change[s, i, j]*cost[s] for s in structures))
#         end
        
        # Economic output is based on sum of buildings there
        @constraint(m, ecoOut[i, j] == sum(objMap[s, i, j]*eco_out[s] for s in structures))
#         end
        
        # Attractiveness is based on sum of buildings there
        @constraint(m, attrac[i, j] == sum(objMap[s, i, j]*attract[s] for s in structures))
#         end
    end
end

@constraint(m, sum(objMap[:hous, i, j] for i = 1:M, j = 1:N) >= pop/5)
# end
@constraint(m, sum(objMap[:scho, i, j] for i = 1:M, j = 1:N) >= pop/50)
# end


# Objective
#         @objective(m, Max, sum(change))
@objective(m, Max, sum(ecoOut) + sum(attrac))

solve(m)
                    
println(getobjectivevalue(m))
println(budget - sum(getvalue(spending)))
                    
opt_map = getvalue(objMap)
printMap(opt_map)

438.0
0.0
offi	manu	offi	manu	offi	offi	offi	offi	offi	manu	
shop	offi	offi	manu	offi	offi	hous	shop	manu	offi	
hous	hous	hous	hous	hous	manu	scho	offi	manu	offi	
hous	hous	hous	manu	hous	shop	hous	offi	hous	hous	
offi	offi	hous	offi	hous	scho	offi	shop	hous	hous	
offi	hous	offi	hous	offi	hous	offi	manu	hous	scho	
offi	scho	hous	manu	shop	hous	shop	offi	manu	hous	
hous	hous	hous	hous	hous	hous	offi	offi	manu	manu	
shop	hous	hous	offi	hous	shop	hous	hous	shop	hous	
shop	shop	hous	offi	hous	shop	offi	hous	manu	hous	
